# Calculate outcomes

This notebook calculates outcomes based on travel times from Lower Super Output Areas (LSOAs) to nearest (in time) acute stroke units that provide thrombolysis and/or thrombectomy.

This notebook calculates the outcomes using the stroke outcome model. The following notebooks combine different cohorts of patients to find the results for a mixed population, and then further notebooks plot the results.

## Plain English summary

We would like to know how travel time to stroke units affects patient outcomes for different parts of England and Wales. We can explore this by keeping the same timings for everything else between the start of the stroke and the beginning of treatment and only adjusting the travel time. For example, a patient who lives 20 minutes further away from the stroke unit than another patient will have an onset to treatment time 20 minutes longer.

In this notebook we look at each Lower Super Output Area (LSOA), a small region of the country that contains a couple of thousand people. For an idea of scale, the city of Exeter is split into around 70 or 80 LSOA. In a more rural location, a larger geographical area must be covered to collect a couple of thousand people and so the LSOAs are larger.

We find a few travel times:
+ the travel time from each LSOA to its nearest stroke unit
+ the travel time from each LSOA to its nearest thrombectomy unit
+ the travel time between each LSOA's nearest stroke unit and its nearest thrombectomy unit, for transfers between hospitals

We then add these to fixed stroke pathway times to calculate a few times to treatment:
+ time to IVT when travelling directly to the nearest stroke unit
+ time to MT after a transfer from the nearest stroke unit to the MT unit
+ time to IVT when travelling directly to the MT unit
+ time to MT when travelling directly to the MT unit

In the UK most of the stroke units will provide thrombolysis but not thrombectomy. Some patients will live nearest to an MT unit and so both of their sets of times will be identical. For most patients though, the time to IVT will be longer if travelling to the MT unit because the unit is further away than their nearest unit. The time to MT is usually longer if a transfer is required than if the patient went directly to the MT unit to start with, partly because the patient spends more time travelling between units and partly because there are delays in getting the patient ready to travel between hospitals.

We can then feed these treatment times into the stroke outcome model. This will give us two sets of outcomes for patients in each LSOA in England and Wales: one set for patients who travel directly to the nearest stroke unit, and a second set for patients who travel directly to the MT centre.


## Aims

Calculate outcomes after treatment for these cohorts:
+ nLVO with IVT
+ LVO with IVT only
+ LVO with MT only
+ LVO with both IVT and MT

Calculate outcomes after treatment for these timings:
+ IVT at the nearest stroke unit
+ MT after transfer to the MT unit
+ IVT after redirection to the MT unit
+ MT after redirection to the MT unit

Then save the outcome data to file for later use.

Two sets of outcomes will be saved:
+ population-averaged added utility, shift in mean mRS, proportion with mRS<=2
+ mRS distribution after treatment

## Method

The travel times are calculated from each LSOA centroid to the location of each stroke unit.

Travel times are estimated using Open Street Map[1], accessed by routino [2].

[1] https://www.openstreetmap.org/

[2] https://packages.debian.org/sid/routino

+ load in stroke-map package
+ Calculate travel time from each LSOA to its nearest acute stroke unit and to its nearest unit providing thrombectomy
+ Add on fixed pathway timings (onset to call, ambulance on scene, door-in-door-out etc.) to find time to thrombolysis and time to thrombectomy
+ Feed treatment times into the outcome model to find outcomes for each patient subgroup at those treatment times.

## Load packages

In [1]:
import numpy as np
import pandas as pd
import copy
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
dir_output = 'output'

In [3]:
# Adjust max rows to show
pd.set_option('display.max_rows', 100)

In [4]:
import stroke_maps
import stroke_outcome

import stroke_outcome.outcome_utilities

In [5]:
stroke_maps.__version__

'0.5.1'

In [6]:
stroke_outcome.__version__

'0.1.6'

In [7]:
import stroke_maps.load_data
import stroke_maps.catchment

from stroke_outcome.continuous_outcome import Continuous_outcome

## Global variables to use

In [8]:
# File name for saved results
file_name = 'lsoa_base'

## Load and parse data

### Load and parse hospital data

In [9]:
# Get list of hopsitals and limit to those used in 2022
hospitals = stroke_maps.load_data.ni_stroke_unit_services()

In [10]:
hospitals['use'] = hospitals[['use_ivt', 'use_mt']].max(axis=1)
mask = hospitals['use'] == 1
hospitals = hospitals[mask]

In [11]:
hospitals.head()

,Hospital_name,City,door_to_needle,Fixed,use_ivt,use_mt,use
Postcode,,,,,,,
BT126BA,Royal Victoria,Belfast,43,2.0,1,1,1
BT161RH,Ulster,Belfast,50,NaN,1,0,1
BT412RL,Antrim Area,Antrim,59,NaN,1,0,1
BT476SB,Altnagelvin,Londonderry,68,NaN,1,0,1
BT521HS,Causeway,Coleraine,45,NaN,1,0,1


### Find LSOA-hospital travel time

In [12]:
travel_matrix = stroke_maps.load_data.travel_time_matrix_ni_areas()

# Show the first five rows and columns:
travel_matrix.iloc[:5, :5]

,BT126BA,BT161RH,BT358DR,BT412RL,BT476SB
from_postcode,,,,,
N00000001,31.400562,37.852665,64.223641,19.032658,79.895643
N00000002,24.677730,37.852665,61.926859,23.292278,83.200000
N00000003,24.677730,37.852665,63.077279,20.471335,80.991077
N00000004,30.080264,42.862547,66.504588,24.677730,83.200000
N00000005,32.710168,40.372700,66.504588,16.088440,76.591575


In [13]:
# Identify closest IVT unit
ivt_hospitals = list(hospitals[hospitals['use_ivt'] == 1].index)
df_catchment_ivt = stroke_maps.catchment.find_nearest_unit(travel_matrix[ivt_hospitals])

df_catchment_ivt.head(3)

,unit_travel_time,unit_postcode
from_postcode,,
N00000001,19.032658,BT412RL
N00000002,23.292278,BT412RL
N00000003,20.471335,BT412RL


In [14]:
# Identify closest MT unit
mt_hospitals = list(hospitals[hospitals['use_mt'] == 1].index)
df_catchment_mt = stroke_maps.catchment.find_nearest_unit(travel_matrix[mt_hospitals])

df_catchment_mt.head(3)

,unit_travel_time,unit_postcode
from_postcode,,
N00000001,31.400562,BT126BA
N00000002,24.677730,BT126BA
N00000003,24.677730,BT126BA


### Find inter-hospital travel time

In [15]:
df_transfer = stroke_maps.catchment.calculate_transfer_units(hospitals, use_northern_ireland=True)

df_transfer.head(3)

,transfer_unit_travel_time,transfer_unit_postcode
Postcode,,
BT126BA,0.000000,BT126BA
BT161RH,19.032658,BT126BA
BT412RL,28.748618,BT126BA


### Gather travel time information

In [16]:
# Copy over IVT unit info:
df_pathway = df_catchment_ivt.copy()
df_pathway = df_pathway.rename(columns={
    'unit_travel_time': 'closest_ivt_time',
    'unit_postcode': 'closest_ivt_unit'
})

# Merge in MT unit info:
df_pathway = pd.merge(
    df_pathway, df_catchment_mt,
    left_index=True, right_index=True, how='left'
)
df_pathway = df_pathway.rename(columns={
    'unit_travel_time': 'closest_mt_time',
    'unit_postcode': 'closest_mt_unit'
})

# Merge in transfer unit info:
df_pathway = pd.merge(
    df_pathway, df_transfer,
    left_on='closest_ivt_unit', right_index=True, how='left'
)
df_pathway = df_pathway.rename(columns={
    'transfer_unit_travel_time': 'transfer_mt_time',
    'transfer_unit_postcode': 'transfer_mt_unit'
})

# Is a transfer required?
df_pathway['mt_transfer_required'] = (
    df_pathway['closest_ivt_unit'] != df_pathway['closest_mt_unit'])

### Add on stroke pathway timings

In [18]:
onset_to_ambulance_arrival = 60
arrival_to_ivt = 30
arrival_to_mt = 90
net_operational_delay_to_mt_for_transfer = 60

In [19]:
# Drip-and-ship:
df_pathway['ivt_drip_ship'] = (
    onset_to_ambulance_arrival +
    df_pathway['closest_ivt_time'] +
    arrival_to_ivt
)
df_pathway['mt_drip_ship'] = (   
    onset_to_ambulance_arrival +
    df_pathway['closest_ivt_time'] + 
    # The times in the following line are only added on if a transfer is required:
    ((net_operational_delay_to_mt_for_transfer + df_pathway['transfer_mt_time']) * df_pathway['mt_transfer_required']) +
    arrival_to_mt
)

# Mothership:
df_pathway['ivt_mothership'] = (
    onset_to_ambulance_arrival +
    df_pathway['closest_mt_time'] +
    arrival_to_ivt
)
df_pathway['mt_mothership'] = (
    onset_to_ambulance_arrival +
    df_pathway['closest_mt_time'] +
    arrival_to_mt
)

In [20]:
# Show df_pathway
df_pathway.head().T

from_postcode,N00000001,N00000002,N00000003,N00000004,N00000005
closest_ivt_time,19.032658,23.292278,20.471335,24.67773,16.08844
closest_ivt_unit,BT412RL,BT412RL,BT412RL,BT412RL,BT412RL
closest_mt_time,31.400562,24.67773,24.67773,30.080264,32.710168
closest_mt_unit,BT126BA,BT126BA,BT126BA,BT126BA,BT126BA
transfer_mt_time,28.748618,28.748618,28.748618,28.748618,28.748618
transfer_mt_unit,BT126BA,BT126BA,BT126BA,BT126BA,BT126BA
mt_transfer_required,True,True,True,True,True
ivt_drip_ship,109.032658,113.292278,110.471335,114.67773,106.08844
mt_drip_ship,257.781276,262.040896,259.219953,263.426348,254.837059
ivt_mothership,121.400562,114.67773,114.67773,120.080264,122.710168


## Calculate outcomes

Calculate the proportion mRS<=2 in the no-treatment population.

In [21]:
mrs_dists, mrs_dists_notes = stroke_outcome.outcome_utilities.import_mrs_dists_from_file()

In [22]:
mrs_dists

,mRS<=0,mRS<=1,mRS<=2,mRS<=3,mRS<=4,mRS<=5,mRS<=6
Stroke type,,,,,,,
pre_stroke_nlvo,0.583,0.746,0.850,0.951,0.993,1.000,1
pre_stroke_lvo,0.408,0.552,0.672,0.838,0.956,1.000,1
no_treatment_lvo,0.050,0.129,0.265,0.429,0.676,0.811,1
no_treatment_nlvo,0.198,0.460,0.580,0.708,0.856,0.918,1
no_effect_nlvo_ivt_deaths,0.196,0.455,0.574,0.701,0.847,0.908,1
no_effect_lvo_ivt_deaths,0.048,0.124,0.255,0.414,0.653,0.783,1
no_effect_lvo_mt_deaths,0.048,0.124,0.255,0.412,0.649,0.779,1
t0_treatment_nlvo_ivt,0.445,0.642,0.752,0.862,0.941,0.967,1
t0_treatment_lvo_ivt,0.140,0.233,0.361,0.522,0.730,0.838,1


In [23]:
no_treatment_nlvo_mrsleq2 = mrs_dists.loc['no_treatment_nlvo', 'mRS<=2']
no_treatment_lvo_mrsleq2 = mrs_dists.loc['no_treatment_lvo', 'mRS<=2']

In [24]:
# Set up outcome model
outcome_model = Continuous_outcome()

In [25]:
patient_cohorts = {
    'nlvo_ivt': {
        'stroke_type_code': 1,
        'ivt_chosen_bool': 1,
        'mt_chosen_bool': 0,
    },
    'lvo_ivt': {
        'stroke_type_code': 2,
        'ivt_chosen_bool': 1,
        'mt_chosen_bool': 0,
    },
    'lvo_ivt_mt': {
        'stroke_type_code': 2,
        'ivt_chosen_bool': 1,
        'mt_chosen_bool': 1,
    },
    'lvo_mt': {
        'stroke_type_code': 2,
        'ivt_chosen_bool': 0,
        'mt_chosen_bool': 1,
    },
}

In [26]:
df_patients = pd.DataFrame()

# Store averaged outcomes in here:
df_results = pd.DataFrame()

# Store mRS distributions in here:
df_mrs = pd.DataFrame()

for model_name in ['drip_ship', 'mothership']:
    df_patients['onset_to_needle_mins'] = df_pathway[f'ivt_{model_name}']
    df_patients['onset_to_puncture_mins'] = df_pathway[f'mt_{model_name}']

    # df_results[f'{model_name}_onset_to_needle_mins'] = df_patients['onset_to_needle_mins'].copy()
    # df_results[f'{model_name}_onset_to_puncture_mins'] = df_patients['onset_to_puncture_mins'].copy()
    
    for cohort_name, cohort_dict in patient_cohorts.items():
        # Assign patient details:
        df_patients['stroke_type_code'] = cohort_dict['stroke_type_code']
        df_patients['ivt_chosen_bool'] = cohort_dict['ivt_chosen_bool']
        df_patients['mt_chosen_bool'] = cohort_dict['mt_chosen_bool']
    
        # Run outcomes:
        outcome_model.assign_patients_to_trial(df_patients)
        
        # Calculate outcomes:
        patient_data_dict, outcomes_by_stroke_type, full_cohort_outcomes = (
            outcome_model.calculate_outcomes())
        
        # Make a copy of the results:
        outcomes_by_stroke_type = copy.copy(outcomes_by_stroke_type)
        full_cohort_outcomes = copy.copy(full_cohort_outcomes)
        
        # Place the relevant results into the results dataframe.
        # Round them to 5 decimal places which should be plenty.
        df_results[f'{model_name}_{cohort_name}_added_utility'] = np.round(
            full_cohort_outcomes['each_patient_utility_shift'], 5)
        df_results[f'{model_name}_{cohort_name}_mean_mrs'] = np.round(
            full_cohort_outcomes['each_patient_mrs_post_stroke'], 5)
        df_results[f'{model_name}_{cohort_name}_mrs_less_equal_2'] = np.round(
            full_cohort_outcomes['each_patient_mrs_dist_post_stroke'][:, 2], 5)
        df_results[f'{model_name}_{cohort_name}_mrs_shift'] = np.round(
            full_cohort_outcomes['each_patient_mrs_shift'], 5)        

        # Calculate the shift in the proportion mRS<=2
        # compared with the no-treatment population.
        if cohort_dict['stroke_type_code'] == 1:
            no_treatment_mrsleq2 = no_treatment_nlvo_mrsleq2
        else:
            no_treatment_mrsleq2 = no_treatment_lvo_mrsleq2

        # Round the results again to avoid floating point errors.
        df_results[f'{model_name}_{cohort_name}_added_mrs_less_equal_2'] = (
            np.round((
                df_results[f'{model_name}_{cohort_name}_mrs_less_equal_2'] -
                no_treatment_mrsleq2
            ), 5))

        # Copy over the mRS distributions:
        # # One list of mRS values per row (patient) in the data.
        outs = full_cohort_outcomes['each_patient_mrs_dist_post_stroke'].copy()
        outs = np.round(outs, 5).tolist()
        df_mrs[[f'{model_name}_{cohort_name}_mrs_dists_{i}' for i in range(7)]] = outs

Merge the outcome results into the travel time dataframe:

In [27]:
df_results.index = df_pathway.index
df_results = pd.concat((df_pathway, df_results), axis='columns')

In [28]:
df_results.head(5).T

from_postcode,N00000001,N00000002,N00000003,N00000004,N00000005
closest_ivt_time,19.032658,23.292278,20.471335,24.67773,16.08844
closest_ivt_unit,BT412RL,BT412RL,BT412RL,BT412RL,BT412RL
closest_mt_time,31.400562,24.67773,24.67773,30.080264,32.710168
closest_mt_unit,BT126BA,BT126BA,BT126BA,BT126BA,BT126BA
transfer_mt_time,28.748618,28.748618,28.748618,28.748618,28.748618
transfer_mt_unit,BT126BA,BT126BA,BT126BA,BT126BA,BT126BA
mt_transfer_required,True,True,True,True,True
ivt_drip_ship,109.032658,113.292278,110.471335,114.67773,106.08844
mt_drip_ship,257.781276,262.040896,259.219953,263.426348,254.837059
ivt_mothership,121.400562,114.67773,114.67773,120.080264,122.710168


In [29]:
df_mrs.index = df_pathway.index
df_mrs = pd.concat((df_pathway, df_mrs), axis='columns')

In [30]:
df_mrs.head(5).T

from_postcode,N00000001,N00000002,N00000003,N00000004,N00000005
closest_ivt_time,19.032658,23.292278,20.471335,24.67773,16.08844
closest_ivt_unit,BT412RL,BT412RL,BT412RL,BT412RL,BT412RL
closest_mt_time,31.400562,24.67773,24.67773,30.080264,32.710168
closest_mt_unit,BT126BA,BT126BA,BT126BA,BT126BA,BT126BA
transfer_mt_time,28.748618,28.748618,28.748618,28.748618,28.748618
transfer_mt_unit,BT126BA,BT126BA,BT126BA,BT126BA,BT126BA
mt_transfer_required,True,True,True,True,True
ivt_drip_ship,109.032658,113.292278,110.471335,114.67773,106.08844
mt_drip_ship,257.781276,262.040896,259.219953,263.426348,254.837059
ivt_mothership,121.400562,114.67773,114.67773,120.080264,122.710168


### Save results to file

In [31]:
df_results.to_csv(os.path.join(dir_output, 'cohort_outcomes.csv'))

In [32]:
df_mrs.to_csv(os.path.join(dir_output, 'cohort_mrs_dists.csv'))

Calculate summary stats for the outcomes:

In [33]:
summary = np.round(df_results.mean(axis=0).T, 5)
summary.to_csv(os.path.join(dir_output, 'cohort_outcomes_summary.csv'), index_label='lsoa')
summary

closest_ivt_time                                 20.16358
closest_mt_time                                  41.00128
transfer_mt_time                                 31.46894
mt_transfer_required                              0.66806
ivt_drip_ship                                   110.16358
mt_drip_ship                                    241.71628
ivt_mothership                                  131.00128
mt_mothership                                   191.00128
drip_ship_nlvo_ivt_added_utility                  0.11579
drip_ship_nlvo_ivt_mean_mrs                       1.64269
drip_ship_nlvo_ivt_mrs_less_equal_2               0.70532
drip_ship_nlvo_ivt_mrs_shift                     -0.63731
drip_ship_nlvo_ivt_added_mrs_less_equal_2         0.12532
drip_ship_lvo_ivt_added_utility                   0.05802
drip_ship_lvo_ivt_mean_mrs                        3.35037
drip_ship_lvo_ivt_mrs_less_equal_2                0.32805
drip_ship_lvo_ivt_mrs_shift                      -0.28963
drip_ship_lvo_

## Conclusion

We have calculated all of the outcome data for the separate cohorts and for both the drip-and-ship and mothership scenarios.

The next notebooks combine the cohorts to find the average results for a mixed population.